# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
cities_file = "C:/Users/Victor M Diaz/Resources/API/Homework/WeatherPy/cities.csv"

cities_df = pd.read_csv(cities_file)

cities_df.head()


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Yellowknife,90,CA,1638334538,83,62.4560,-114.3525,-12.59,4.00
1,Rikitea,0,PF,1638334538,76,-23.1203,-134.9692,76.41,20.69
2,Bluff,9,NZ,1638334539,74,-46.6000,168.3333,68.81,18.41
3,Saint-Philippe,88,RE,1638334539,74,-21.3585,55.7679,81.95,8.21
4,East London,75,ZA,1638334347,94,-33.0153,27.9116,63.54,3.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
cities_df = cities_df.dropna()
Humidity = cities_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
Location = cities_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

Heat_map = gmaps.heatmap_layer(Location, weights = Humidity, 
                                 dissipating=False, max_intensity=max(cities_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(Heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
# Narrow down the DataFrame to find your ideal weather condition.
city_df = cities_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
city_df = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 50)]

# Wind speed less than 10 mph.
city_df = city_df.loc[city_df["Wind Speed"] < 5]

# Zero cloudiness.
city_df = city_df.loc[city_df["Cloudiness"] == 0]

# Drop any rows with null values
city_df = city_df.dropna()

city_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
108,Coquimbo,0,CL,1638334396,82,-29.9533,-71.3436,55.13,2.30
145,Castro,0,BR,1638334370,78,-24.7911,-50.0119,56.57,2.89
175,Artesia,0,US,1638334589,59,33.8658,-118.0831,78.06,1.50
202,El Salvador,0,SV,1638334501,83,13.8333,-88.9167,63.19,3.83
247,Havelock,0,US,1638334611,70,34.8791,-76.9013,53.94,1.01
295,Kentau,0,KZ,1638334627,29,43.5206,68.5094,53.46,4.07
297,Salta,0,AR,1638334627,65,-24.7859,-65.4117,73.69,3.44
307,Gujar Khan,0,PK,1638334630,24,33.2558,73.3026,65.43,2.64
346,San Ramon,0,US,1638334642,61,37.7799,-121.9780,65.25,2.73
355,Morro Bay,0,US,1638334645,59,35.3658,-120.8499,64.78,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = city_df


params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
             }


for index, row in hotel_df.iterrows():
    # find the latitude and longitude
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{latitude},{longitude}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        
    print("-------------------Next Hotel Search------------")

Retrieving Results for Index 108: Coquimbo.
Closest hotel is Punto de encuentro.
-------------------Next Hotel Search------------
Retrieving Results for Index 145: Castro.
Closest hotel is CHACARA BAILLY.
-------------------Next Hotel Search------------
Retrieving Results for Index 175: Artesia.
Closest hotel is Days Inn & Suites by Wyndham Artesia.
-------------------Next Hotel Search------------
Retrieving Results for Index 202: El Salvador.
Closest hotel is Chico's Ranch.
-------------------Next Hotel Search------------
Retrieving Results for Index 247: Havelock.
Closest hotel is Sherwood Motel.
-------------------Next Hotel Search------------
Retrieving Results for Index 295: Kentau.
Closest hotel is Safary.
-------------------Next Hotel Search------------
Retrieving Results for Index 297: Salta.
Closest hotel is Hotel Alejandro I.
-------------------Next Hotel Search------------
Retrieving Results for Index 307: Gujar Khan.
Closest hotel is Tanzeem e Islami Halqa Gujar Khan.
-----

In [28]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
108,Coquimbo,0,CL,1638334396,82,-29.9533,-71.3436,55.13,2.30,Punto de encuentro
145,Castro,0,BR,1638334370,78,-24.7911,-50.0119,56.57,2.89,CHACARA BAILLY
175,Artesia,0,US,1638334589,59,33.8658,-118.0831,78.06,1.50,Days Inn & Suites by Wyndham Artesia
202,El Salvador,0,SV,1638334501,83,13.8333,-88.9167,63.19,3.83,Chico's Ranch
247,Havelock,0,US,1638334611,70,34.8791,-76.9013,53.94,1.01,Sherwood Motel
295,Kentau,0,KZ,1638334627,29,43.5206,68.5094,53.46,4.07,Safary
297,Salta,0,AR,1638334627,65,-24.7859,-65.4117,73.69,3.44,Hotel Alejandro I
307,Gujar Khan,0,PK,1638334630,24,33.2558,73.3026,65.43,2.64,Tanzeem e Islami Halqa Gujar Khan
346,San Ramon,0,US,1638334642,61,37.7799,-121.9780,65.25,2.73,San Ramon Marriott
355,Morro Bay,0,US,1638334645,59,35.3658,-120.8499,64.78,1.01,Inn at Morro Bay


In [30]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""



In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))